In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import clean_data
import pandas as pd
import modeling_2 as model
import numpy as np
import matplotlib.pyplot as plt
import feature_selection as fs
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

The baseline model for changes in median earnings assumes a constant wage growth rate. This rate is a gross approximation since entry-level jobs after college may change at a different rate than the entire industry. Furthermore, different employement sectors and geographic areas will have different earnings growth rates.

In [41]:
WAGE_GROWTH_RATE = 1.03

In [49]:
merged_df = clean_data.runAll()
merged_df_no_id = merged_df.drop('UNITID', axis=1)

In [208]:
#test_baseline is starting year of median earnings for each institution
test_baseline = merged_df[['UNITID','MD_EARN_WNE_P6', 'Year']]
test_baseline = test_baseline[test_baseline['Year'].isin(TEST_YEARS)]
test_baseline.reset_index()
test_baseline.sort_values(by=['UNITID', 'Year'])
test_baseline = test_baseline.rename(columns={"MD_EARN_WNE_P6": "TEST_MD_EARN_WNE_P6"})

#model generates estimated earnings for each institution by applying WAGE_GROWTH_RATE to previous year
target = 'MD_EARN_WNE_P6'
merged_df_baseline_2013 = pd.DataFrame(merged_df[merged_df['Year'] == 2012]) 
merged_df_baseline_2013 = merged_df_baseline_2013[['MD_EARN_WNE_P6', 'Year', 'UNITID']]
merged_df_baseline_2013[target] = merged_df_baseline_2013[target].apply(lambda x: x + np.log(WAGE_GROWTH_RATE))
merged_df_baseline_2013['Year'] = 2013
merged_df_baseline_2013.reset_index()

merged_df_baseline_2014 = merged_df_baseline_2013.copy()
merged_df_baseline_2014[target] = merged_df_baseline_2013[target].apply(lambda x: x + np.log(WAGE_GROWTH_RATE))
merged_df_baseline_2014['Year'] = 2014
merged_df_baseline_2014.reset_index()

merged_baseline = pd.concat([merged_df_baseline_2013, merged_df_baseline_2014])
merged_baseline.sort_values(by=['UNITID', 'Year'])
merged_baseline=merged_baseline.rename(columns={"MD_EARN_WNE_P6": "PRED_MD_EARN_WNE_P6"})
merged_baseline.columns

Index(['PRED_MD_EARN_WNE_P6', 'Year', 'UNITID'], dtype='object')

In [210]:
complete_df = pd.merge(test_baseline, merged_baseline, how='left', on=['UNITID', 'Year'])
complete_df = complete_df.dropna()

**Model Evaluation**

In [220]:
mean_squared_error(complete_df['TEST_MD_EARN_WNE_P6'], complete_df['PRED_MD_EARN_WNE_P6'])

0.004897730972596517

In [226]:
mean_absolute_error(complete_df['TEST_MD_EARN_WNE_P6'].apply(lambda x: np.exp(x)), \
                                complete_df['PRED_MD_EARN_WNE_P6'].apply(lambda x: np.exp(x)))

1313.7888732394367